<a href="https://colab.research.google.com/github/Hadiasemi/Data301/blob/main/Copy_of_DATA_301_Lab_4B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Song Lyrics Generator

In this lab, you will scrape a website to get lyrics of songs by your favorite artist. Then, you will train a model called a Markov chain on these lyrics so that you can generate a song in the style of your favorite artist.

# Question 1. Scraping Song Lyrics

Find a web site that has lyrics for several songs by your favorite artist. Scrape the lyrics into a Python list called `lyrics`, where each element of the list represents the lyrics of one song.

**Tips:**
- Find a web page that has links to all of the songs, like [this one](https://www.azlyrics.com/s/stevemillerband.html). [_Note:_ It appears that `azlyrics.com` blocks web scraping, so you may have to find a different lyrics web site.] Then, you can scrape this page, extract the hyperlinks, and issue new HTTP requests to each hyperlink to get each song. 
- Use `time.sleep()` to stagger your HTTP requests so that you do not get banned by the website for making too many requests.

In [ ]:
import requests
import time

from bs4 import BeautifulSoup

In [ ]:
response = requests.get("https://www.songlyrics.com/soulfly-lyrics/")

In [ ]:
response.text

In [ ]:
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
import re
lyrics = []

tables = soup.find_all("tbody")

for song in tables[0].find_all('tr'):
  # find all the links
  link = song.find('a',  href=True)
  # check to make sure not be None
  if link != None:
    # go to each link and get the lyric page
    get_lyrics = requests.get(link['href'])
    soup = BeautifulSoup(get_lyrics.content, "html.parser")
    lyric = soup.find('p', id ='songLyricsDiv').text
    lyric = lyric.replace('\n' ,' <N> ').replace('[^\w\s]','').replace('\r', '').replace('\\', '')
    # make sure the lyric is available
    if 'We do not have the lyrics for' not in lyric:
      lyrics.append(lyric)
total_lyrics = []

pattern = r'\[[^\]]*\]'
for d in lyrics:
  total_lyrics.append(re.sub(pattern, '', d))

total_lyrics

In [ ]:
print(total_lyrics[0])

We the land of carnival and murder <N> Bring the drums and the sounding of thunder <N> Oh my people we got to get stronger <N> With the spirit of war in quilombos <N>  <N> Jungle people, jungle <N> Never underestimate our power <N> Jungle people, jungle <N> Never underestimate our power <N>  <N> Ah, loco <N> Bring the shit <N>  <N> Bring our roots we're under the sun <N> From in the jungle and from the slums <N> Our freedom we bring with this song <N> Never forget where you came from <N>  <N> Jungle people, jungle <N> Never underestimate our power <N> Jungle people, jungle <N> Never underestimate our power <N>  <N> Bumba bumba <N> Bumba bumba <N> Bumba bumba <N>  <N> Oxossi open my way <N> I'm gonna tell the world what you say <N> I know God making me a way <N> We pray, we pray everyday <N>  <N> Jungle people, jungle <N> Never underestimate our power <N> Jungle people, jungle <N> Never underestimate our power <N>  <N> Bumba bumba <N> Bumba bumba <N> Bumba bumba <N>  <N> Ele e caboclo e

`pickle` is a Python library that serializes Python objects to disk so that you can load them in later.

In [ ]:
import pickle
pickle.dump(total_lyrics, open("lyrics.pkl", "wb"))

# Question 2. Unigram Markov Chain Model

You will build a Markov chain for the artist whose lyrics you scraped in Question 1. Your model will process the lyrics and store the word transitions for that artist. The transitions will be stored in a dict called `chain`, which maps each word to a list of "next" words.

For example, if your song was ["The Joker" by the Steve Miller Band](https://www.youtube.com/watch?v=dV3AziKTBUo), `chain` might look as follows:

```
chain = {
    "some": ["people", "call", "people"],
    "call": ["me", "me", "me"],
    "the": ["space", "gangster", "pompitous", ...],
    "me": ["the", "the", "Maurice"],
    ...
}
```

Besides words, you should include a few additional states in your Markov chain. You should have `"<START>"` and `"<END>"` states so that we can keep track of how songs are likely to begin and end. You should also include a state called `"<N>"` to denote line breaks so that you can keep track of where lines begin and end. It is up to you whether you want to include normalize case and strip punctuation.

So for example, for ["The Joker"](https://www.azlyrics.com/lyrics/stevemillerband/thejoker.html), you would add the following to your chain:

```
chain = {
    "<START>": ["Some", ...],
    "Some": ["people", ...],
    "people": ["call", ...],
    "call": ["me", ...],
    "me": ["the", ...],
    "the": ["space", ...],
    "space": ["cowboy,", ...],
    "cowboy,": ["yeah", ...],
    "yeah": ["<N>", ...],
    "<N>": ["Some", ..., "Come"],
    ...,
    "Come": ["on", ...],
    "on": ["baby", ...],
    "baby": ["and", ...],
    "and": ["I'll", ...],
    "I'll": ["show", ...],
    "show": ["you", ...],
    "you": ["a", ...],
    "a": ["good", ...],
    "good": ["time", ...],
    "time": ["<END>", ...],
}
```

Your chain will be trained on not just one song, but by all songs by your artist.

In [ ]:
res = []
lyric_ = []
for i, lyric in enumerate(total_lyrics):
  v = ("<START> " + lyric + " <END>").split()

  lyric_.append(v)

  
lyric_


In [ ]:
def train_markov_chain(lyrics):
    """
    Args:
      - lyrics: a list of strings, where each string represents
                the lyrics of one song by an artist.
    
    Returns:
      A dict that maps a single word ("unigram") to a list of
      words that follow that word, representing the Markov
      chain trained on the lyrics.
    """

    chain = {"<START>": []}
    for lyric in lyrics:
        # YOUR CODE HERE
        for i, word in enumerate(lyric):
          if word != "<END>":
            if word not in chain.keys():
              chain[word] = [lyric[i+1]]
            else:
              chain[word].append(lyric[i+1]) if lyric[i+1] not in chain[word] else 1
          

    return chain
train_markov_chain(lyric_)

In [ ]:
# Load the pickled lyrics object that you created in Question 1.
import pickle
lyrics = pickle.load(open("lyrics.pkl", "rb"))

# Call the function you wrote above.
chain = train_markov_chain(lyric_)

# What words tend to start a song (i.e., what words follow the <START> tag?)
print(chain["<START>"])

# What words tend to begin a line (i.e., what words follow the line break tag?)
print(chain["<N>"][:20])

# import random
# print(random.choice(chain["<START>"]))

['We', 'In', 'Up', 'Jump', 'Fall', 'Bumba,', 'Pablo', 'Um', 'The', 'Die,', '*Zumbi', 'Here', 'I', 'What', 'Fire,', 'When', 'Pressure', 'This', '<END>', 'On', 'Um,', 'Off', '<N>', 'My', 'Impaled', 'Blood,', '*we', 'Counter', 'Shame', 'Sacred', 'Judas…', 'Ð', 'Live', 'Zumbi', 'Lightning', 'Speak', 'Disciple', 'Lost', 'One,', 'Time', 'Yo,', 'Grow', 'Feels', 'Every', 'Manananananaue', 'Vamos', 'Maggots', 'Clean', 'War', 'Material', 'Monday,', 'Spreading', 'You', 'All', 'No', 'Dark', 'Bezouro,', 'Carved', 'Oh,', 'Whoever', 'Murderer', 'Why', 'Now', 'Extreme...', 'Fighting', 'Earth', 'So', 'Welcome', 'Doomsday', 'Keep', 'Wings', 'Born', 'Feel', 'Mulambo', "I'm", 'Vulture', 'Porrada', 'Instrumental', 'Prophecies', 'Namaste', '*', 'Quilombo', 'Told', 'SoulFly', 'No,', 'Brainwashed', 'Chaos', 'INTRO:', 'Roots', 'Master', 'And', 'Fuck', 'Your', 'Oh', 'Prophecy', 'live', 'Sorry,', 'Intro]', 'Featuring', 'Jumpdafuckup!', 'Fire...', 'Judas...', 'Bloody', 'Brethren,', 'Fight', 'Rome', "We're", 'Well

Now, let's generate new lyrics using the Markov chain you constructed above. To do this, we'll begin at the `"<START>"` state and randomly sample a word from the list of words that follow `"<START>"`. Then, at each step, we'll randomly sample the next word from the list of words that followed each current word. We will continue this process until we sample the `"<END>"` state. This will give us the complete lyrics of a randomly generated song!

You may find the `random.choice()` function helpful for this question.

In [ ]:
import random

def generate_new_lyrics(chain):
    """
    Args:
      - chain: a dict representing the Markov chain,
               such as one generated by generate_new_lyrics()
    
    Returns:
      A string representing the randomly generated song.
    """
    
    # a list for storing the generated words
    words = []
    # generate the first word
    word = random.choice(chain["<START>"])
    
    # YOUR CODE HERE
    while word != "<END>":
        words.append(word)
        word = random.choice(chain[word])

    
    
    # join the words together into a string with line breaks
    lyrics = " ".join(words[:-1])
    return "\n".join(lyrics.split("<N>"))

In [ ]:
print(generate_new_lyrics(chain))

We the land do this 
 And everything turns inside out 
 You can't mask pain, my pain, our


# Question 3. Bigram Markov Chain Model

Now you'll build a more complex Markov chain that uses the last _two_ words (or bigram) to predict the next word. Now your dict `chain` should map a _tuple_ of words to a list of words that appear after it.

As before, you should also include tags that indicate the beginning and end of a song, as well as line breaks. That is, a tuple might contain tags like `"<START>"`, `"<END>"`, and `"<N>"`, in addition to regular words. So for example, for ["The Joker"](https://www.azlyrics.com/lyrics/stevemillerband/thejoker.html), you would add the following to your chain:

```
chain = {
    (None, "<START>"): ["Some", ...],
    ("<START>", "Some"): ["people", ...],
    ("Some", "people"): ["call", ...],
    ("people", "call"): ["me", ...],
    ("call", "me"): ["the", ...],
    ("me", "the"): ["space", ...],
    ("the", "space"): ["cowboy,", ...],
    ("space", "cowboy,"): ["yeah", ...],
    ("cowboy,", "yeah"): ["<N>", ...],
    ("yeah", "<N>"): ["Some", ...],
    ("time", "<N>"): ["Come"],
    ...,
    ("<N>", "Come"): ["on", ...],
    ("Come", "on"): ["baby", ...],
    ("on", "baby"): ["and", ...],
    ("baby", "and"): ["I'll", ...],
    ("and", "I'll"): ["show", ...],
    ("I'll", "show"): ["you", ...],
    ("show", "you"): ["a", ...],
    ("you", "a"): ["good", ...],
    ("a", "good"): ["time", ...],
    ("good", "time"): ["<END>", ...],
}
```

In [ ]:
def train_markov_chain(lyrics):
    """
    Args:
      - lyrics: a list of strings, where each string represents
                the lyrics of one song by an artist.
    
    Returns:
      A dict that maps a tuple of 2 words ("bigram") to a list of
      words that follow that bigram, representing the Markov
      chain trained on the lyrics.
    """
    chain = {(None, "<START>"): []}
    for lyric in lyrics:
        # YOUR CODE HERE
        lines = lyric.split()
        bigram = (None, "<START>")
        for word in lines:
            chain[bigram].append(word)
            bigram = (bigram[1], word)
            if bigram not in chain:
                chain[bigram] = []
        chain[bigram].append("<END>")

    return chain

In [ ]:
# Load the pickled lyrics object that you created in Question 1.
import pickle
lyrics = pickle.load(open("lyrics.pkl", "rb"))

# Call the function you wrote above.
chain = train_markov_chain(lyrics)

# What words tend to start a song (i.e., what words follow the <START> tag?)
print(chain[(None, "<START>")])

['We', 'In', 'Up', 'Jump', 'Fall', 'Bumba,', 'Pablo', 'Um', 'The', 'Die,', 'Jump', '*Zumbi', 'Here', 'I', 'What', 'Fire,', 'When', 'Pressure', 'This', '<END>', 'We', 'On', 'Um,', 'Off', '<N>', 'My', 'This', 'Impaled', 'Blood,', '*we', 'Counter', 'Shame', 'Sacred', 'I', 'I', 'Judas…', 'Ð', 'Live', 'Zumbi', 'Lightning', 'Speak', 'Disciple', 'Lost', 'One,', 'Time', 'Yo,', 'Grow', 'Feels', 'Every', 'What', 'Pressure', 'Manananananaue', 'Vamos', 'Maggots', 'My', 'Clean', 'War', 'Material', 'Monday,', 'Spreading', 'I', 'You', '<N>', 'All', 'I', 'No', 'You', 'Dark', 'Bezouro,', 'Carved', '<END>', 'Oh,', 'Whoever', 'No', 'Murderer', 'Why', 'Now', 'Extreme...', '<END>', 'Fighting', 'Earth', '<END>', 'The', 'Zumbi', 'So', 'Welcome', 'Doomsday', 'Keep', 'We', 'My', '<END>', 'Wings', 'You', 'Born', 'Feel', 'Mulambo', "I'm", 'The', 'Whoever', 'Vulture', 'Now', 'Porrada', 'Instrumental', '<END>', '<END>', 'I', 'Prophecies', 'Namaste', '<END>', '*', "I'm", 'Quilombo', 'Told', 'Grow', 'The', 'What', '

Now, let's generate new lyrics using the Markov chain you constructed above. To do this, we'll begin at the `(None, "<START>")` state and randomly sample a word from the list of words that follow this bigram. Then, at each step, we'll randomly sample the next word from the list of words that followed the current bigram (i.e., the last two words). We will continue this process until we sample the `"<END>"` state. This will give us the complete lyrics of a randomly generated song!

In [ ]:
import random

def generate_new_lyrics(chain):
    """
    Args:
      - chain: a dict representing the Markov chain,
               such as one generated by generate_new_lyrics()
    
    Returns:
      A string representing the randomly generated song.
    """
    
    # a list for storing the generated words
    words = []
    # generate the first word
    words.append(random.choice(chain[(None, "<START>")]))
    
    bigram = ("<START>",words[0])
    word = random.choice(chain[bigram])

    while word != "<END>":
        words.append(word)
        bigram = (words[-2], words[-1])
        word = random.choice(chain[bigram])
    
    # join the words together into a string with line breaks
    lyrics = " ".join(words[:-1])
    return "\n".join(lyrics.split("<N>"))

In [ ]:
print(generate_new_lyrics(chain))


 The seven gates of Namtar fell 
 Jfk shot in the calunga and in your brain 
 
 This corrosion creeps 
 And limb amputations 
 
 Come around, come around 
 Now motivation comes from beyond 
 Bloodbath 
 Tyrants 
 Bastards 
 
 I hit the bottom 
 You know we can't pretend 
 Here we go 
 Another day, another hour 
 Here we go 
 
 What we want to burn 
 Starting to burn up the scene 
 Trust no soul 
 
 Against all odds we carry on 
 
 Here we go 
 I'd rather die on my feet 
 Here we go 
 It's All We Wanna Be 
 Watch us


# Question 4. Analysis

Compare the quality of the lyrics generated by the unigram model (in Question 2) and the bigram model (in Question 3). Which model seems to generate more reasonable lyrics? Can you explain why? What do you see as the advantages and disadvantages of each model?

In my perspective the bigram has more quilty to it compare to the unigram. Also, the bigram generate longer length compare to unigram. As result, the bigram generate more reseanable model compare to unigram. The issue that I can see in both is they both include lots of repitation in the lyrics generated. 

## Submission Instructions

- Restart this notebook and run the cells from beginning to end:
  - Go to Runtime > Restart and Run All.
- Download the notebook:
  - Go to File > Download > Download .ipynb.
- Submit your notebook file to the assignment on Canvas.
